In [1]:
import os
%pwd

'/media/kirti/Dev/DeepLearning/Project/E2E/ChestCancerDetection/research'

In [2]:
os.chdir("../")
%pwd

'/media/kirti/Dev/DeepLearning/Project/E2E/ChestCancerDetection'

In [3]:
import dagshub
dagshub.init(repo_owner='Kra09-kp', repo_name='E2EChestCancerDetection', mlflow=True)

Accessing as Kra09-kp

Initialized MLflow to track repo "Kra09-kp/E2EChestCancerDetection"

Repository Kra09-kp/E2EChestCancerDetection initialized!

In [4]:
import mlflow
import torch

In [5]:
from cnnClassifier.components.prepare_base_model import PrepareBaseModel

In [6]:
model_path = "Model/final_model.pth"

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path 
    testing_data: str
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [ ]:
class configManager:
    def __init__(self,
                 config_path: Path = CONFIG_FILE_PATH,
                 params_path: Path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            model_path = Path("artifacts/train_model/best_model.pth"),
            testing_data = "artifacts/data_ingestion/Data",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/Kra09-kp/E2EChestCancerDetection.mlflow", 
            params_image_size=self.params.IMAGE_SIZE[:2],
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    

In [18]:
import torch
import torch.nn as nn
import torchvision
import os
from torchvision.models import VGG16_Weights
from torch.utils import data
from torchvision.transforms import v2 as T
from tqdm import tqdm 
from urllib.parse import urlparse

In [19]:


class ModelEvaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def test_generator(self):
        self._transform = T.Compose([
            T.Resize(self.config.params_image_size),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
        ])

        self.test_dataset = torchvision.datasets.ImageFolder(
            root=os.path.join(self.config.testing_data, "test"),
            transform=self._transform
        )

        self.test_loader = data.DataLoader(
            dataset=self.test_dataset,
            batch_size=self.config.params_batch_size,
            shuffle=False
        )



    @staticmethod
    def load_model(config: EvaluationConfig):

        model_path = config.model_path
        classes = config.all_params['CLASSES']
        freeze_all = config.all_params['FREEZE_ALL']
        freeze_till = config.all_params['FREEZE_TILL']

        # return classes
        model = torchvision.models.vgg16(weights=None)

        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False

        elif freeze_till is not None:
            for param in list(model.parameters())[:freeze_till]:
                param.requires_grad = False

        # Modify the classifier
        model.classifier = nn.Sequential(
            nn.Linear(25088, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, classes)  # for 4-class classification
        )

    
        state_dict = torch.load(model_path,map_location=torch.device('cpu'))
        model.load_state_dict(state_dict)
        print(type(model))
        return model
    

    def evaluation(self):
        self.test_generator()
        self.model = self.load_model(self.config)
        len_test_dataset = len(self.test_dataset)
        self.model.to(self.device)
        self.model.eval()

        accuracy = 0.0
        loss = 0.0
        with torch.no_grad():
            loop = tqdm(self.test_loader, total=len(self.test_loader), desc="Evaluating Model")
            for images, labels in loop:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                criterion = nn.CrossEntropyLoss()
                # Calculate loss and accuracy
                loss += criterion(outputs, labels).item() * images.size(0)
                preds = torch.argmax(outputs, dim=1)
                accuracy += (preds == labels).sum().item()
                loop.set_description("Evaluation")
                loop.set_postfix(loss=loss, acc=accuracy)
                # For testing purposes, we break after the first batch
                break 

        val_accuracy = accuracy / len_test_dataset * 100
        val_loss = loss / len_test_dataset

        self.score = {
            "model_accuracy": val_accuracy,
            "model_loss": val_loss
        }
        print(self.score)
        return self.score
    

    def save_score(self):
        save_json(
            path = Path("scores.json"),
            data = self.score
        )

        print(f"Score saved at {Path('scores.json').resolve()}")

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.set_experiment("Checking Model")

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("model_accuracy", self.score["model_accuracy"])
            mlflow.log_metric("model_loss", self.score["model_loss"])
            
            print(tracking_url_type_store)  # debugging
            
            if tracking_url_type_store != "file":
                # Local MLflow — use model registry
                mlflow.pytorch.log_model(self.model, "model", registered_model_name="ChestCancerDetectionModel")
            else:
                # Remote MLflow (like DagsHub) — no registry
                mlflow.pytorch.log_model(self.model, "model")
            
        

In [20]:
try:
    config = configManager()
    eval_config = config.get_evaluation_config()
    model_eval = ModelEvaluation(eval_config)
    model_eval.evaluation()
    model_eval.log_into_mlflow()

except Exception as e:  
    raise e

[2025-07-18 19:53:06,500|(INFO)| File: common | Message: Created directory: artifacts]


/home/kirti/anaconda3/envs/deeplearning/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


/tmp/ipykernel_15911/1400308713.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path,map_location=torch.device('cpu'))


<class 'torchvision.models.vgg.VGG'>


Evaluation:   0%|          | 0/40 [00:58<?, ?it/s, acc=6, loss=8.27]


{'model_accuracy': 1.9047619047619049, 'model_loss': 0.02624742417108445}
[2025-07-18 19:56:46,547|(WARNING)| File: connectionpool | Message: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /Kra09-kp/E2EChestCancerDetection.mlflow/api/2.0/mlflow/experiments/get-by-name?experiment_name=Checking+Model]
https
[2025-07-18 20:05:37,543|(WARNING)| File: connectionpool | Message: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'TimeoutError('The write operation timed out')': /Kra09-kp/E2EChestCancerDetection.mlflow/api/2.0/mlflow-artifacts/artifacts/72db5ed5d54f4bdfb5c17184ab9c7296/9eb146ee28cc4f568ee763a6ea312929/artifacts/model/data/model.pth]


Successfully registered model 'ChestCancerDetectionModel'.
2025/07/18 20:11:01 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ChestCancerDetectionModel, version 1
Created version '1' of model 'ChestCancerDetectionModel'.
